In [1]:
import pandas as pd
import sqlite3
import os

--- Execução do Notebook Load to Database ---

In [2]:
# --- Configuração ---
path_silver = '../data/silver/dados_limpos.csv'
path_gold = '../data/gold/'
db_path = '../data/pipeline_agricultura.db'

# Conectar ao banco SQLite (cria se não existir)
conn = sqlite3.connect(db_path)
print(f"Conexão com banco de dados '{db_path}' estabelecida.")

Conexão com banco de dados '../data/pipeline_agricultura.db' estabelecida.


In [3]:
# ==========================================
# 1. CARREGAR DADOS LIMPOS (SILVER)
# ==========================================
print("\nCarregando Dados Limpos (Silver)...")
try:
    # O df_limpo já deve ter as colunas climáticas renomeadas do Bronze/Silver
    df_limpo = pd.read_csv(path_silver)
    print(f"Dados limpos carregados: {df_limpo.shape}")
except FileNotFoundError:
    print(f"Erro: Arquivo '{path_silver}' não encontrado.")
    conn.close()
    exit()


Carregando Dados Limpos (Silver)...
Dados limpos carregados: (55682, 31)


In [4]:
# ==========================================
# 2. CRIAR TABELAS DE DIMENSÃO (Modelagem)
# ==========================================



# --- Dimensão Região ---
if 'State District' in df_limpo.columns:
    dim_regiao = df_limpo[['State District']].drop_duplicates().sort_values('State District').reset_index(drop=True)
    # Salva no DB, usando o novo índice como chave primária (id_regiao)
    dim_regiao.to_sql('dim_regiao', conn, if_exists='replace', index_label='id_regiao')
    print(f" - Tabela 'dim_regiao' criada com {len(dim_regiao)} registros.")

    # Mapeamento para substituição de chave
    map_regiao = dim_regiao.reset_index().rename(
    columns={'index': 'id_regiao'}
    ).set_index('State District')['id_regiao'].to_dict()
    
else:
     print(" - AVISO: Coluna 'State District' não encontrada em df_limpo. 'dim_regiao' não criada.")
     map_regiao = {}

 - AVISO: Coluna 'State District' não encontrada em df_limpo. 'dim_regiao' não criada.


In [5]:
# --- Dimensão Cultura (Crop) ---
if 'Crop' in df_limpo.columns:
    dim_cultura = df_limpo[['Crop']].drop_duplicates().sort_values('Crop').reset_index(drop=True)
    dim_cultura.to_sql('dim_cultura', conn, if_exists='replace', index_label='id_cultura')
    print(f" - Tabela 'dim_cultura' criada com {len(dim_cultura)} registros.")

    # Mapeamento para substituição de chave
    map_cultura = dim_cultura.reset_index().rename(
        columns={'index': 'id_cultura'}
    ).set_index('Crop')['id_cultura'].to_dict()
else:
     print(" - AVISO: Coluna 'Crop' não encontrada em df_limpo. 'dim_cultura' não criada.")
     map_cultura = {}

 - AVISO: Coluna 'Crop' não encontrada em df_limpo. 'dim_cultura' não criada.


In [6]:
# --- Dimensão Temporada (Season) ---
if 'Season' in df_limpo.columns:
    dim_temporada = df_limpo[['Season']].drop_duplicates().sort_values('Season').reset_index(drop=True)
    dim_temporada.to_sql('dim_temporada', conn, if_exists='replace', index_label='id_temporada')
    print(f" - Tabela 'dim_temporada' criada com {len(dim_temporada)} registros.")

    # Mapeamento para substituição de chave
    map_temporada = dim_temporada.reset_index().rename(
        columns={'index': 'id_temporada'}
    ).set_index('Season')['id_temporada'].to_dict()
else:
     print(" - AVISO: Coluna 'Season' não encontrada em df_limpo. 'dim_temporada' não criada.")
     map_temporada = {}

 - AVISO: Coluna 'Season' não encontrada em df_limpo. 'dim_temporada' não criada.


In [7]:
# ==========================================
# 3. IMPLEMENTAR SUBSTITUIÇÃO DE CHAVES (MELHORIA CRÍTICA)
# ==========================================

if map_regiao:
    df_limpo['id_regiao'] = df_limpo['State District'].map(map_regiao)
    df_limpo.drop(columns=['State District'], inplace=True) # Remove colunas de texto para a chave natural
    print(" - Chave 'id_regiao' adicionada e colunas de texto removidas.")

In [8]:
if map_cultura:
    df_limpo['id_cultura'] = df_limpo['Crop'].map(map_cultura)
    df_limpo.drop(columns=['Crop'], inplace=True)
    print(" - Chave 'id_cultura' adicionada e colunas de texto removidas.")

In [9]:
if map_temporada:
    df_limpo['id_temporada'] = df_limpo['Season'].map(map_temporada)
    df_limpo.drop(columns=['Season'], inplace=True)
    print(" - Chave 'id_temporada' adicionada e colunas de texto removidas.")

In [10]:
# ==========================================
# 4. CARREGAR TABELA FATO (com chaves estrangeiras)
# ==========================================
df_limpo.to_sql('fato_producao', conn, if_exists='replace', index=False)
print(f"Tabela 'fato_producao' criada: {len(df_limpo)} registros.")

Tabela 'fato_producao' criada: 55682 registros.


In [11]:
# ==========================================
# 5. CARREGAR DADOS AGREGADOS (GOLD)
# ==========================================

arquivos_gold = [
    'producao_anual_cultura.csv',
    'desempenho_regiao_cultura.csv',
    'analise_sazonal_clima.csv',
    'tendencia_anual_rendimento.csv',
    'benchmark_regional_rendimento.csv',
    'perfil_climatico_regiao_cultura.csv',
    'volatilidade_rendimento_regiao.csv'
]

Não é mais necessário o gold_rename_dict, pois a renomeação foi feita no Bronze/Silver

In [12]:
for arquivo in arquivos_gold:
    caminho_arquivo = os.path.join(path_gold, arquivo)
    try:
        df_gold = pd.read_csv(caminho_arquivo)

        # O nome da tabela será o nome do arquivo sem a extensão .csv
        nome_tabela = os.path.splitext(arquivo)[0]

        df_gold.to_sql(nome_tabela, conn, if_exists='replace', index=False)
        print(f" - Tabela Gold '{nome_tabela}' criada com {len(df_gold)} registros.")

    except FileNotFoundError:
        print(f" - AVISO: Arquivo Gold '{arquivo}' não encontrado. Tabela ignorada.")
    except Exception as e:
        print(f" - ERRO ao carregar '{arquivo}': {e}")

 - Tabela Gold 'producao_anual_cultura' criada com 88 registros.
 - Tabela Gold 'desempenho_regiao_cultura' criada com 2451 registros.
 - Tabela Gold 'analise_sazonal_clima' criada com 20 registros.
 - Tabela Gold 'tendencia_anual_rendimento' criada com 88 registros.
 - Tabela Gold 'benchmark_regional_rendimento' criada com 2451 registros.
 - Tabela Gold 'perfil_climatico_regiao_cultura' criada com 2451 registros.
 - Tabela Gold 'volatilidade_rendimento_regiao' criada com 2278 registros.


In [13]:
# ==========================================
# 6. VERIFICAR TABELAS CRIADAS
# ==========================================
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()

In [14]:
print(f"Tabelas no banco de dados '{db_path}':")
for tabela in tabelas:
    print(f" - {tabela[0]}")

Tabelas no banco de dados '../data/pipeline_agricultura.db':
 - dim_regiao
 - dim_cultura
 - dim_temporada
 - fato_producao
 - producao_anual_cultura
 - desempenho_regiao_cultura
 - analise_sazonal_clima
 - tendencia_anual_rendimento
 - benchmark_regional_rendimento
 - perfil_climatico_regiao_cultura
 - volatilidade_rendimento_regiao


In [15]:
conn.close()
print("\nDados carregados com sucesso no banco de dados!")


Dados carregados com sucesso no banco de dados!
